<h2>Description:</h2> 

The dataset contains the clients information collected through campaign by portueguese bank institution, so that we would know which clients the bank should target so that the clients will subscribe to the long term deposit scheme of the bank.

<br>
The dataset consist of 20 attributes and one class label which is the binary valued attribute indicating whether the client has subscribe the scheme or not.


<h2>Features: Clients Data</h2>

1 - age (numeric)<br>

2 - job : type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")<br>

3 - marital : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)<br>

4 - education (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")<br>

5 - default: has credit in default? (categorical: "no","yes","unknown")<br>

6 - housing: has housing loan? (categorical: "no","yes","unknown")<br>

7 - loan: has personal loan? (categorical: "no","yes","unknown")

<h2>Features related to the last contact of the current campaign</h2>

8 - contact: contact communication type (categorical: "cellular","telephone")<br> 

9 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")<br>

10 - day_of_week: last contact day of the week (categorical: "mon","tue","wed","thu","fri")<br>

11 - duration: last contact duration, in seconds (numeric).

<h2>Other Features</h2>

12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)<br>

13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)<br>

14 - previous: number of contacts performed before this campaign and for this client (numeric)<br>

15 - poutcome: outcome of the previous marketing campaign (categorical: "failure","nonexistent","success")

<h2>Social and economic context attributes</h2>

16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)<br>

17 - cons.price.idx: consumer price index - monthly indicator (numeric)     <br>

18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)     <br>

19 - euribor3m: euribor 3 month rate - daily indicator (numeric)<br>

20 - nr.employed: number of employees - quarterly indicator (numeric)

<h2>Output variable (desired target):</h2>

21 - y - has the client subscribed a term deposit? (binary: "yes","no")<br>

In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report

In [22]:
df = pd.read_csv("bank-additional-full.csv",";")

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
dtypes: float64(5), int64(5), object(11)
memory usa

In [24]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [35]:
df['y'].replace('no',0,inplace = True)
df['y'].replace('yes',1,inplace = True)

In [36]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0


In [37]:
df_modified = df.copy()
for i in range(2,8):
    df_modified = df_modified[df_modified.iloc[:,i] != 'unknown']

In [38]:
print('Before removing unknown valuess: ',df.shape[0])
print('After removing unknown valuess: ',df_modified.shape[0])
print('% of unknown: ', (df.shape[0]-df_modified.shape[0])/df.shape[0]*100,'%')

Before removing unknown valuess:  41188
After removing unknown valuess:  30604
% of unknown:  25.696804894629505 %


In [117]:
UnderSampler = RandomUnderSampler('majority')

X = df_modified.iloc[:,:-1]
y = df_modified.iloc[:,-1]

X_re,y_re = UnderSampler.fit_resample(X,y)

In [118]:
X_re = pd.DataFrame(X_re)
y_re = pd.DataFrame(y_re)

df_re = pd.concat([X_re,y_re],axis=1)

df_re.columns = df.columns

In [119]:
transform_list=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
df_re = pd.get_dummies(df_re,columns=transform_list,prefix=transform_list)

In [123]:
# Ordering Features
cols_list = list(df_re.columns.values) 
cols_list.pop(cols_list.index('y')) 
df_re = df_re[cols_list+['y']]

In [124]:
X = df_re.iloc[:,:-1].values
y = df_re.iloc[:,-1]

sc_X = StandardScaler()
X = sc_X.fit_transform(X)

labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [125]:
RF = RandomForestClassifier()

RF = RF.fit(X,y)

features_list_RF = list(zip(cols_list,RF.feature_importances_))
features_list_RF.sort(key=lambda x: x[1],reverse = True)

In [126]:
LF = LogisticRegression(penalty='l2')
LF.fit(X,y)

top_features = 10
coefficients = LF.coef_[0]
top_index = abs(coefficients).argsort()[-10:][::-1]

features_list_LF =[]
for index in top_index:
    features_list_LF.append((cols_list[index],coefficients[index]))

features_list_LF.sort(key=lambda x: x[1],reverse = True)

In [127]:
features_list_RF[:10]

[('duration', 0.341683669312856),
 ('euribor3m', 0.06914979981648332),
 ('age', 0.06074099084376998),
 ('nr.employed', 0.04774235708244612),
 ('cons.price.idx', 0.044672221489700785),
 ('emp.var.rate', 0.03881097974105002),
 ('campaign', 0.034242785501005964),
 ('poutcome_success', 0.03196246664593537),
 ('cons.conf.idx', 0.02525025604262874),
 ('contact_telephone', 0.018883710540072284)]

In [128]:
features_list_LF

[('duration', 2.34980644147102),
 ('cons.price.idx', 1.2374134014069995),
 ('euribor3m', 0.9562165451971127),
 ('month_mar', 0.3964402962999242),
 ('month_aug', 0.32418706126724106),
 ('nr.employed', 0.24715699436863775),
 ('month_nov', -0.18334793352179637),
 ('month_may', -0.30337859690529223),
 ('pdays', -0.4649538298191298),
 ('emp.var.rate', -3.295024521050912)]

In [141]:
top_features_cols = []
for index in top_index:
    top_features_cols.append(cols_list[index])

In [145]:
df_re_top_features = df_re[top_features_cols]
X = df_re_top_features.iloc[:,:-1].values

sc_X = StandardScaler()
X = sc_X.fit_transform(X)

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [146]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33, random_state = 0)

In [149]:
RF.fit(X_train,y_train)
y_pred_RF = RF.predict(X_test)

In [150]:
LF.fit(X_train,y_train)
y_pred_LF = LF.predict(X_test)

In [152]:
print(classification_report(y_test,y_pred_RF))

             precision    recall  f1-score   support

          0       0.86      0.85      0.85      1286
          1       0.85      0.86      0.85      1273

avg / total       0.85      0.85      0.85      2559



In [153]:
print(classification_report(y_test,y_pred_LF))

             precision    recall  f1-score   support

          0       0.88      0.85      0.86      1286
          1       0.85      0.88      0.87      1273

avg / total       0.87      0.87      0.87      2559

